# LSTM - Generic Framework - Investigation

In [1]:
#####################################################################################
run_models = True
run_k_fold = False
overleaf_metrics = False
print_all = False
is_deadline = False
#####################################################################################


#####################################################################################
# ['DNN', 'LSTM']
model_name = 'LSTM'
#####################################################################################


#####################################################################################
k_fold_n_splits = 5
#####################################################################################



#####################################################################################
arousal_signals = ['PP']
# arousal_signals = ['PP', 'BR']
# arousal_signals = ['PP_HR_BR']
# arousal_signals = ['PP', 'HR', 'BR']
# arousal_signals = ['PP', 'HR', 'BR', 'PP_HR', 'HR_BR', 'PP_BR', 'PP_HR_BR']
#####################################################################################



#####################################################################################
%run us_common_functions.ipynb
#####################################################################################




#####################################################################################
# study_combinations_to_run = list(study_combinations.keys())
# study_combinations_to_run = list(study_combinations.keys())[:4]
# study_combinations_to_run = list(study_combinations.keys())[4:]

# study_combinations_to_run = ['sim1', 'sim1___sim2___tt1___office_tasks']
# study_combinations_to_run = ['sim2', 'tt1', 'office_tasks', 'sim1___sim2', 'sim1___sim2___tt1']
# study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']
# study_combinations_to_run = ['sim1', 'tt1']
# study_combinations_to_run = ['sim1']
#### study_combinations_to_run.remove(deadline_study)
####################################################################################




####################################################################################
### study_combinations_to_run = all_study_combinations[:-1]
### study_combinations_to_run = ['sim1___sim2___tt1___office_tasks']

study_combinations, all_study_combinations = get_study_combinations(studies)
study_combinations_to_run = all_study_combinations
print(study_combinations_to_run)
####################################################################################




# #####################################################################################
# # For Deadline Study - Without BR
# #####################################################################################
# studies.append(deadline_study)
# study_combinations, all_study_combinations = get_study_combinations(studies)
# study_combinations_to_run = studies
# print(study_combinations_to_run)

# arousal_signals = ['PP', 'HR', 'PP_HR']
# model_features.remove('BR_Mean')
# model_features.remove('BR_SD')

# is_deadline=True
# ####################################################################################










####################################################################################
# ground_truth_method = 'hist'
ground_truth_method = 'ecdf'
####################################################################################




####################################################################################
epochs = 2  # 10
batch_size = 32
verbose = 1
####################################################################################


In [2]:
####################################################################################
# Initialization
####################################################################################
model_metrics = {}
overleaf_final_tables = defaultdict(lambda : None)
study_combination_model_metrics = defaultdict(lambda : None)

# figure_path = all_studies_fig_dir
# plot_name = None



# data_6_rolling_w15_5_scaled.csv
# data_6_rolling_w30_5_scaled.csv
# data_6_rolling_w60_5_scaled.csv
dnn_data_file = 'data_6_rolling_w30_5_scaled.csv'

dnn_window_size = dnn_data_file[-15:-11]
####################################################################################

In [4]:
# all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file)
all_studies_df = pd.read_csv(all_studies_data_dir + dnn_data_file).iloc[:2000,]
all_studies_df.rename({'Perinasal_Log_Window': 'Perinasal_Window'}, axis=1, inplace=True)
all_studies_df.shape

(2000, 58)

In [5]:
all_studies_df.head(2)

,Study_Name,Subject,Study_Subject,Treatment,Age,NASA_Effort,NASA_Frustration,NASA_Mental,NASA_Performance,NASA_Physical,NASA_Temporal,NASA_Total,Perinasal_Mean_NN,Perinasal_Median_NN,Perinasal_SD_NN,Perinasal_SS_NN,Perinasal_Window_NN,Perinasal_Mean,Perinasal_Median,Perinasal_SD,Perinasal_SS,Perinasal_Window,Heart_Mean_NN,Heart_Median_NN,Heart_SD_NN,Heart_SS_NN,Heart_Window_NN,HR_Mean,HR_Median,HR_SD,HR_SS,Heart_Window,Breathing_Mean_NN,Breathing_Median_NN,Breathing_SD_NN,Breathing_SS_NN,Breathing_Window_NN,BR_Mean,BR_Median,BR_SD,BR_SS,Breathing_Window,PP_Arousal_Mode_Hist,HR_Arousal_Mode_Hist,BR_Arousal_Mode_Hist,PP_Arousal_Mode,HR_Arousal_Mode,BR_Arousal_Mode,PP_HR_Arousal_Mode,PP_BR_Arousal_Mode,HR_BR_Arousal_Mode,PP_HR_BR_Arousal_Mode,Gender,Gender_Male,PP_HR_Arousal_Mode_Hist,PP_BR_Arousal_Mode_Hist,HR_BR_Arousal_Mode_Hist,PP_HR_BR_Arousal_Mode_Hist
0,sim1,2,sim1_2,CD,-0.886388,0.96301,0.332856,18.0,-0.204407,0.548524,0.367759,77.0,-5.282175,-5.297421,0.037635,837.082385,"-5.218846387983004, -5.214028019882206, -5.212...",-0.704197,-0.043052,0.102389,0.064272,"0.0355228579351853, 0.0403412260359834, 0.0419...",90.540000,90.4,1.436615,245984.60,"90.2, 91.6, 92.2, 92.2, 91.6, 90.4, 89.2, 88.8...",0.919132,3.768182,-0.513985,518.068554,"3.5681818181818272, 4.968181818181819, 5.56818...",12.054667,12.1,0.903494,4383.1224,"10.8, 10.88, 10.92, 10.98, 11.02, 11.08, 11.12...",-1.088477,-4.366682,0.178627,607.649078,"-5.666681818181818, -5.586681818181818, -5.546...",relaxed,relaxed,relaxed,relaxed,neutral,relaxed,relaxed,relaxed,neutral,relaxed,0.691472,0,relaxed,relaxed,relaxed,relaxed
1,sim1,2,sim1_2,CD,-0.886388,0.96301,0.332856,18.0,-0.204407,0.548524,0.367759,77.0,-5.305937,-5.307565,0.033802,844.622019,"-5.229819391933348, -5.24046285310951, -5.2518...",-0.864359,-0.053195,-0.021171,0.112911,"0.0245498539848414, 0.0139063928086793, 0.0025...",89.553333,90.2,2.234330,240738.76,"90.4, 89.2, 88.8, 88.6, 88.6, 89.4, 90.2, 90.4...",0.698460,3.568182,0.052955,400.832190,"3.76818181818183, 2.5681818181818272, 2.168181...",12.553333,12.5,0.965521,4754.6200,"11.08, 11.12, 11.18, 11.22, 11.32, 11.4, 11.56...",-0.948112,-3.966682,0.281929,486.463558,"-5.386681818181819, -5.346681818181819, -5.286...",relaxed,stressed,relaxed,relaxed,neutral,relaxed,neutral,relaxed,neutral,neutral,0.691472,0,relaxed,relaxed,relaxed,relaxed


In [16]:
all_studies_df.columns

Index(['Study_Name', 'Subject', 'Study_Subject', 'Treatment', 'Age',
       'NASA_Effort', 'NASA_Frustration', 'NASA_Mental', 'NASA_Performance',
       'NASA_Physical', 'NASA_Temporal', 'NASA_Total', 'Perinasal_Mean',
       'Perinasal_Median', 'Perinasal_SD', 'Perinasal_SS',
       'Perinasal_Mean_10s', 'Perinasal_Median_10s', 'Perinasal_SD_10s',
       'Perinasal_SS_10s', 'Perinasal_Window', 'HR_Mean', 'HR_Median', 'HR_SD',
       'HR_SS', 'Heart_Mean_10s', 'Heart_Median_10s', 'Heart_SD_10s',
       'Heart_SS_10s', 'Heart_Window', 'BR_Mean', 'BR_Median', 'BR_SD',
       'BR_SS', 'Breathing_Mean_10s', 'Breathing_Median_10s',
       'Breathing_SD_10s', 'Breathing_SS_10s', 'Breathing_Window',
       'PP_Arousal_Mode_Hist', 'HR_Arousal_Mode_Hist', 'BR_Arousal_Mode_Hist',
       'PP_Arousal_Mode', 'HR_Arousal_Mode', 'BR_Arousal_Mode',
       'PP_HR_Arousal_Mode', 'PP_BR_Arousal_Mode', 'HR_BR_Arousal_Mode',
       'PP_HR_BR_Arousal_Mode', 'Gender', 'Gender_Male',
       'PP_HR_Arousal_Mod

In [17]:
all_studies_df = process_ts_features(all_studies_df)
all_studies_df = create_categorical_labels(all_studies_df)

In [18]:
start_time = time.time()

# study_combinations_to_run
# all_study_combinations
for study_combination in study_combinations_to_run:   
    
    train_studies = study_combinations[study_combination]['Train']
    test_studies = study_combinations[study_combination]['Test']
    # print(train_studies, test_studies, study_combination)
    
    train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
    # print(all_studies_df.shape, train_df.shape)
    
    model_metrics = {}

    
    for arousal_signal in arousal_signals:
        
        train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
        # if ground_truth_method == 'ecdf': 
        # train_df = train_df[train_df[arousal_signal+'_Arousal_Mode_Cat'] != -1]
        
        try:
            print(decorator + '\n' + decorator)
            print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
            print(decorator + '\n' + decorator + '\n')


            # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
            model_metrics[arousal_signal] = {
                'K_Fold': copy.deepcopy(all_metrics),
                'Train': copy.deepcopy(all_metrics),
                'Test': {}
            }

            # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
            # print(train_df.shape)
            
            model = get_trained_dnn_model(train_df, arousal_signal)


            # for i, test_study in enumerate(test_studies):
            for j in range(1, len(test_studies)+1):
                for subset in itertools.combinations(test_studies, j):
              
                    test_study_subset = list(subset)
                    test_study_str = '___'.join(test_study_subset)
                    
                    if test_study_str in all_study_combinations:
                        # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
                        print('\n------------------------ Test Study: ' + test_study_str.upper() + ' ------------------------')

                        model_metrics[arousal_signal]['Test'][test_study_str.upper()] = copy.deepcopy(all_metrics)
                        # test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
                        test_df = all_studies_df.copy()[all_studies_df.Study_Name == test_study_str]
                        # if ground_truth_method == 'ecdf': 
                        # test_df = test_df[test_df[arousal_signal+'_Arousal_Mode_Cat'] != -1]
                        test_dnn_model(model, test_study_str.upper(), test_df, arousal_signal)

                        metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics, k_fold=run_k_fold), ignore_index = True)

            if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics, k_fold=False), ignore_index = True)
    
        except Exception as e:
            print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))
    
    # pprint.pprint(model_metrics)
    # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
    study_combination_model_metrics[study_combination] = model_metrics
    
    
display(metrics_df)
metrics_file_name = 'dnn_deadline_____' if is_deadline else 'dnn_____'
metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + metrics_file_name + dnn_window_size + '_v4_' + get_date_time() + '.csv', sep=',', index=False)


end_time = time.time()

----------------------------------------------------
----------------------------------------------------
Train Studies:  SIM1 & Modality: PP
----------------------------------------------------
----------------------------------------------------

Epoch 1/10
204/204 [==============================] - 13s 27ms/step - loss: 1.5189 - accuracy: 0.6963 - auc_63: 0.8807 - f1_m: 0.5606 - precision_m: 0.8507 - recall_m: 0.4407 - val_loss: 0.0540 - val_accuracy: 0.7964 - val_auc_63: 0.9442 - val_f1_m: 0.7387 - val_precision_m: 0.9420 - val_recall_m: 0.6154
Epoch 2/10
204/204 [==============================] - 2s 8ms/step - loss: 0.0413 - accuracy: 0.8396 - auc_63: 0.9518 - f1_m: 0.8134 - precision_m: 0.9539 - recall_m: 0.7198 - val_loss: 0.0355 - val_accuracy: 0.8930 - val_auc_63: 0.9653 - val_f1_m: 0.8831 - val_precision_m: 0.9355 - val_recall_m: 0.8419
Epoch 3/10
204/204 [==============================] - 2s 9ms/step - loss: 0.0318 - accuracy: 0.8770 - auc_63: 0.9669 - f1_m: 0.8626 - precisi

,Model,Arousal_Signal,Train_Study,Test_Study,Train_Accuracy,Test_Accuracy,Train_AUC,Test_AUC,Train_F1,Test_F1,Train_Recall,Test_Recall,Train_Precision,Test_Precision,Train_Specificity,Test_Specificity
0,LSTM,PP,sim1,sim2,0.94,0.61,0.99,0.63,0.94,0.61,0.94,0.61,0.94,0.61,0.97,0.63
1,LSTM,PP,sim1,tt1,0.94,0.72,0.99,0.82,0.94,0.71,0.94,0.72,0.94,0.72,0.97,0.81
2,LSTM,PP,sim1,office_tasks,0.94,0.63,0.99,0.72,0.94,0.63,0.94,0.63,0.94,0.63,0.97,0.65
3,LSTM,PP,sim1,deadline_study,0.94,0.64,0.99,0.70,0.94,0.64,0.94,0.64,0.94,0.64,0.97,0.58
4,LSTM,HR,sim1,sim2,0.82,0.56,0.94,0.52,0.82,0.55,0.82,0.56,0.85,0.57,0.95,0.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,LSTM,BR,sim1___sim2___tt1___office_tasks,deadline_study,0.92,0.70,0.99,0.76,0.92,0.69,0.92,0.70,0.92,0.72,0.99,0.84
248,LSTM,PP_HR,sim1___sim2___tt1___office_tasks,deadline_study,0.95,0.82,0.99,0.88,0.95,0.81,0.95,0.82,0.95,0.83,0.98,0.92
249,LSTM,HR_BR,sim1___sim2___tt1___office_tasks,deadline_study,0.95,0.68,0.99,0.70,0.95,0.67,0.95,0.68,0.95,0.71,0.98,0.86
250,LSTM,PP_BR,sim1___sim2___tt1___office_tasks,deadline_study,0.97,0.72,1.00,0.86,0.97,0.72,0.97,0.72,0.97,0.72,0.99,0.75


In [19]:
# loss: 0.0208 - accuracy: 0.9001 - f1_m: 0.8761 - precision_m: 0.9773 - recall_m: 0.8009
# val_loss: 0.0325 - val_accuracy: 0.8527 - val_f1_m: 0.8304 - vals_precision_m: 0.9588 - val_recall_m: 0.7407

In [20]:
modeling_time = end_time - start_time
print("--- Total Time %s mins (%s hours) ---" % (modeling_time//60, modeling_time//3600))

--- Total Time 54.0 mins (0.0 hours) ---


In [21]:
# root_path = add_path(all_studies_data_dir, metrics_dir) 
# files = ['metrics_lstm_v0.csv', 'metrics_lstm_v1.csv']
# final_file_name = 'metrics_lstm_all.csv'

# concat_df(root_path, files, final_file_name)

In [22]:
# start_time = time.time()

# for study_combination in study_combinations_to_run:
    
#     train_studies = study_combinations[study_combination]['Train']
#     test_studies = study_combinations[study_combination]['Test']
#     # print(train_studies, test_studies, study_combination)
    
#     train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
#     # print(all_studies_df.shape, train_df.shape)
    
#     model_metrics = {}

    
#     for arousal_signal in arousal_signals:
        
#         train_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(train_studies)]
#         if ground_truth_method == 'ecdf': 
#             train_df = train_df[train_df[arousal_signal+'_Arousal_Mode_Cat'] != -1]
        
#         try:
#             print(decorator + '\n' + decorator)
#             print('Train Studies: ', study_combination.upper() + ' & Modality: ' + arousal_signal)
#             print(decorator + '\n' + decorator + '\n')


#             # Ref: https://stackoverflow.com/questions/3975376/understanding-dict-copy-shallow-or-deep
#             model_metrics[arousal_signal] = {
#                 'K_Fold': copy.deepcopy(all_metrics),
#                 'Train': copy.deepcopy(all_metrics),
#                 'Test': {}
#             }

#             # print(train_df[arousal_signal+'_Arousal_Mode_Cat'].unique())
#             # print(train_df.shape)
            
#             model = get_trained_dnn_model(train_df, arousal_signal)


#             # for i, test_study in enumerate(test_studies):
#             for j in range(1, len(test_studies)+1):
#                 for subset in itertools.combinations(test_studies, j):
              
#                     test_study_subset = list(subset)
#                     test_study_str = '___'.join(test_study_subset).upper()
#                     # print('\n------------------------ Test Study' + str(i+1) + ': ' + test_study.upper() + ' ------------------------')
#                     print('\n------------------------ Test Study: ' + test_study_str + ' ------------------------')
 
#                     model_metrics[arousal_signal]['Test'][test_study_str] = copy.deepcopy(all_metrics)
#                     test_df = all_studies_df.copy()[all_studies_df.Study_Name.isin(test_study_subset)]
#                     if ground_truth_method == 'ecdf': 
#                         test_df = test_df[test_df[arousal_signal+'_Arousal_Mode_Cat'] != -1]
#                     test_dnn_model(model, test_study_str, test_df, arousal_signal)

#                     metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, test_study_str, model_metrics, k_fold=run_k_fold), ignore_index = True)

#             if not test_studies: metrics_df = metrics_df.append(get_test_study_metrics_df(arousal_signal, study_combination, None, model_metrics), ignore_index = True)
    
#         except Exception as e:
#             print("An exception occurred for %s %s --> %s" % (study_combination, arousal_signal, e))
    
#     # pprint.pprint(model_metrics)
#     # overleaf_final_tables[study_combination] = get_overleaf_table(study_combination, model_metrics)
#     study_combination_model_metrics[study_combination] = model_metrics
    
    
# display(metrics_df)
# metrics_df.to_csv(add_path(all_studies_data_dir, metrics_dir) + 'lstm_' + dnn_window_size + '_' + get_date_time() + '.csv', sep=',', index=False)


# end_time = time.time()